In [2]:
# Intialization
import os
import sys

os.environ["SPARK_HOME"] = "/home/talentum/spark"
os.environ["PYLIB"] = os.environ["SPARK_HOME"] + "/python/lib"
# In below two lines, use /usr/bin/python2.7 if you want to use Python 2
os.environ["PYSPARK_PYTHON"] = "/usr/bin/python3.6" 
os.environ["PYSPARK_DRIVER_PYTHON"] = "/usr/bin/python3"
sys.path.insert(0, os.environ["PYLIB"] +"/py4j-0.10.7-src.zip")
sys.path.insert(0, os.environ["PYLIB"] +"/pyspark.zip")

# NOTE: Whichever package you want mention here.
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0 pyspark-shell' 
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-avro_2.11:2.4.0 pyspark-shell'
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0,org.apache.spark:spark-avro_2.11:2.4.3 pyspark-shell'
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0,org.apache.spark:spark-avro_2.11:2.4.0 pyspark-shell'pyspark-shell'

In [3]:
#Entrypoint 2.x
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Spark SQL basic example").enableHiveSupport().getOrCreate()

# On yarn:
# spark = SparkSession.builder.appName("Spark SQL basic example").enableHiveSupport().master("yarn").getOrCreate()
# specify .master("yarn")

sc = spark.sparkContext

In [4]:
df_csv = spark.read.csv("autos.csv", header=True, inferSchema=True) #Reading Data From CSV file

In [5]:
df_csv.printSchema() #printing Schema

root
 |-- index: string (nullable = true)
 |-- dateCrawled: string (nullable = true)
 |-- name: string (nullable = true)
 |-- seller: string (nullable = true)
 |-- offerType: string (nullable = true)
 |-- price: string (nullable = true)
 |-- abtest: string (nullable = true)
 |-- vehicleType: string (nullable = true)
 |-- yearOfRegistration: string (nullable = true)
 |-- gearbox: string (nullable = true)
 |-- powerPS: string (nullable = true)
 |-- model: string (nullable = true)
 |-- kilometer: integer (nullable = true)
 |-- monthOfRegistration: string (nullable = true)
 |-- fuelType: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- notRepairedDamage: string (nullable = true)
 |-- dateCreated: string (nullable = true)
 |-- nrOfPictures: integer (nullable = true)
 |-- postalCode: string (nullable = true)
 |-- lastSeen: timestamp (nullable = true)



In [6]:
df_csv.count() #getting total Row Count

371824

In [7]:
df_csv_duplicates=df_csv.dropDuplicates() #Droping the Duplicates
df_csv_duplicates.count()

371824

In [8]:
df_csv.columns #Getting the Column name


['index',
 'dateCrawled',
 'name',
 'seller',
 'offerType',
 'price',
 'abtest',
 'vehicleType',
 'yearOfRegistration',
 'gearbox',
 'powerPS',
 'model',
 'kilometer',
 'monthOfRegistration',
 'fuelType',
 'brand',
 'notRepairedDamage',
 'dateCreated',
 'nrOfPictures',
 'postalCode',
 'lastSeen']

In [9]:
df_csv.select('price').describe().show() 

+-------+------------------+
|summary|             price|
+-------+------------------+
|  count|            371823|
|   mean|17295.124087885943|
| stddev|  3587958.57312194|
|    min|                 0|
|    max|              test|
+-------+------------------+



In [10]:
df_csv.filter("vehicleType is NULL").show()

+-----+-------------------+--------------------+------+---------+-----+-------+-----------+------------------+-------+-------+--------+---------+-------------------+--------+----------+-----------------+-------------------+------------+----------+-------------------+
|index|        dateCrawled|                name|seller|offerType|price| abtest|vehicleType|yearOfRegistration|gearbox|powerPS|   model|kilometer|monthOfRegistration|fuelType|     brand|notRepairedDamage|        dateCreated|nrOfPictures|postalCode|           lastSeen|
+-----+-------------------+--------------------+------+---------+-----+-------+-----------+------------------+-------+-------+--------+---------+-------------------+--------+----------+-----------------+-------------------+------------+----------+-------------------+
|    0|2016-03-24 11:52:17|          Golf_3_1.6|privat|  Angebot|  480|   test|       null|              1993|manuell|      0|    golf|   150000|                  0|  benzin|volkswagen|           

In [11]:
df_csv=df_csv.drop('nrOfPictures','seller','lastSeen','dateCreated')

In [12]:
df_csv.columns

['index',
 'dateCrawled',
 'name',
 'offerType',
 'price',
 'abtest',
 'vehicleType',
 'yearOfRegistration',
 'gearbox',
 'powerPS',
 'model',
 'kilometer',
 'monthOfRegistration',
 'fuelType',
 'brand',
 'notRepairedDamage',
 'postalCode']

In [13]:
df_csv.select('brand').distinct().show() #Distinct Model of Car

+-------------+
|        brand|
+-------------+
|       jaguar|
|         nein|
|     daihatsu|
|   mitsubishi|
|         null|
|         lada|
|       toyota|
|         seat|
|         saab|
|   land_rover|
|      peugeot|
|     chrysler|
|      citroen|
|         audi|
|mercedes_benz|
|          bmw|
|         jeep|
|       lancia|
|        skoda|
|        rover|
+-------------+
only showing top 20 rows



In [14]:
from pyspark.sql.functions import to_timestamp

In [15]:
df_csv = df_csv.withColumn('dateCrawled', to_timestamp(df_csv["dateCrawled"], "yyyy-MM-dd HH:mm:ss")) 
#Changing the car  price to Given Format 


In [16]:
df_csv.printSchema()

root
 |-- index: string (nullable = true)
 |-- dateCrawled: timestamp (nullable = true)
 |-- name: string (nullable = true)
 |-- offerType: string (nullable = true)
 |-- price: string (nullable = true)
 |-- abtest: string (nullable = true)
 |-- vehicleType: string (nullable = true)
 |-- yearOfRegistration: string (nullable = true)
 |-- gearbox: string (nullable = true)
 |-- powerPS: string (nullable = true)
 |-- model: string (nullable = true)
 |-- kilometer: integer (nullable = true)
 |-- monthOfRegistration: string (nullable = true)
 |-- fuelType: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- notRepairedDamage: string (nullable = true)
 |-- postalCode: string (nullable = true)



In [17]:
df_csv.select('dateCrawled').show(5)

+-------------------+
|        dateCrawled|
+-------------------+
|2016-03-24 11:52:17|
|2016-03-24 10:58:45|
|2016-03-14 12:52:21|
|2016-03-17 16:54:04|
|2016-03-31 17:25:20|
+-------------------+
only showing top 5 rows



In [18]:
from pyspark.sql.functions import to_timestamp, year
df_csv = df_csv.withColumn("dateCrawled", year(df_csv["dateCrawled"]))

In [19]:
df_csv.select('dateCrawled').show(5)

+-----------+
|dateCrawled|
+-----------+
|       2016|
|       2016|
|       2016|
|       2016|
|       2016|
+-----------+
only showing top 5 rows



In [20]:
df_csv.printSchema()

root
 |-- index: string (nullable = true)
 |-- dateCrawled: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- offerType: string (nullable = true)
 |-- price: string (nullable = true)
 |-- abtest: string (nullable = true)
 |-- vehicleType: string (nullable = true)
 |-- yearOfRegistration: string (nullable = true)
 |-- gearbox: string (nullable = true)
 |-- powerPS: string (nullable = true)
 |-- model: string (nullable = true)
 |-- kilometer: integer (nullable = true)
 |-- monthOfRegistration: string (nullable = true)
 |-- fuelType: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- notRepairedDamage: string (nullable = true)
 |-- postalCode: string (nullable = true)



In [21]:
df_csv.select('yearOfRegistration').show(10)

+------------------+
|yearOfRegistration|
+------------------+
|              1993|
|              2011|
|              2004|
|              2001|
|              2008|
|              1995|
|              2004|
|              1980|
|              2014|
|              1998|
+------------------+
only showing top 10 rows



In [22]:
from pyspark.sql.functions import col, when
df_csv = df_csv.withColumn('yearOfRegistration', col('yearOfRegistration').cast("int"))
#Converting year of Registeration to int

In [23]:
df_csv.printSchema()

root
 |-- index: string (nullable = true)
 |-- dateCrawled: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- offerType: string (nullable = true)
 |-- price: string (nullable = true)
 |-- abtest: string (nullable = true)
 |-- vehicleType: string (nullable = true)
 |-- yearOfRegistration: integer (nullable = true)
 |-- gearbox: string (nullable = true)
 |-- powerPS: string (nullable = true)
 |-- model: string (nullable = true)
 |-- kilometer: integer (nullable = true)
 |-- monthOfRegistration: string (nullable = true)
 |-- fuelType: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- notRepairedDamage: string (nullable = true)
 |-- postalCode: string (nullable = true)



In [24]:
df_csv = df_csv.withColumn('Price', col('Price').cast("float")) #Converting Thr Price To float

In [25]:
df_csv.printSchema()

root
 |-- index: string (nullable = true)
 |-- dateCrawled: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- offerType: string (nullable = true)
 |-- Price: float (nullable = true)
 |-- abtest: string (nullable = true)
 |-- vehicleType: string (nullable = true)
 |-- yearOfRegistration: integer (nullable = true)
 |-- gearbox: string (nullable = true)
 |-- powerPS: string (nullable = true)
 |-- model: string (nullable = true)
 |-- kilometer: integer (nullable = true)
 |-- monthOfRegistration: string (nullable = true)
 |-- fuelType: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- notRepairedDamage: string (nullable = true)
 |-- postalCode: string (nullable = true)



In [26]:
df_csv = df_csv.withColumn('monthOfRegistration', col('monthOfRegistration').cast("int")) #ConvertingThe Month Of registeration to int


In [27]:
df_csv.printSchema()

root
 |-- index: string (nullable = true)
 |-- dateCrawled: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- offerType: string (nullable = true)
 |-- Price: float (nullable = true)
 |-- abtest: string (nullable = true)
 |-- vehicleType: string (nullable = true)
 |-- yearOfRegistration: integer (nullable = true)
 |-- gearbox: string (nullable = true)
 |-- powerPS: string (nullable = true)
 |-- model: string (nullable = true)
 |-- kilometer: integer (nullable = true)
 |-- monthOfRegistration: integer (nullable = true)
 |-- fuelType: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- notRepairedDamage: string (nullable = true)
 |-- postalCode: string (nullable = true)



In [28]:
correlation_value2_value3 = df_csv.stat.corr("monthOfRegistration", "Price") #finding  the Correlation between MonthofRegisterationand price
print(f"Correlation between value2 and value3: {correlation_value2_value3}")

Correlation between value2 and value3: -0.0028176063434703284


In [29]:
correlation_value2_value3 = df_csv.stat.corr("dateCrawled", "Price") #finding the Correaltion betwwen the Datecrawled and price
print(f"Correlation between value2 and value3: {correlation_value2_value3}")

Correlation between value2 and value3: 0.00013600453977892399


In [30]:
df_csv.select("monthOfRegistration").distinct().show() #finding the Distict value for Month of Registeration

+-------------------+
|monthOfRegistration|
+-------------------+
|                 12|
|               null|
|                  1|
|                  6|
|                  3|
|                  5|
|                  9|
|                  4|
|                  8|
|                  7|
|                 10|
|                 11|
|                  2|
|                  0|
+-------------------+



In [31]:
df_csv=df_csv.drop('offerType') #only two tyep agebot and Genuch

In [32]:
df_csv.columns

['index',
 'dateCrawled',
 'name',
 'Price',
 'abtest',
 'vehicleType',
 'yearOfRegistration',
 'gearbox',
 'powerPS',
 'model',
 'kilometer',
 'monthOfRegistration',
 'fuelType',
 'brand',
 'notRepairedDamage',
 'postalCode']

In [33]:
zero_count = df_csv.filter(col("Price") == 0).count()
print(zero_count) #filtering the columns where price is 0

10779


In [34]:
df_csv.columns

['index',
 'dateCrawled',
 'name',
 'Price',
 'abtest',
 'vehicleType',
 'yearOfRegistration',
 'gearbox',
 'powerPS',
 'model',
 'kilometer',
 'monthOfRegistration',
 'fuelType',
 'brand',
 'notRepairedDamage',
 'postalCode']

In [35]:
from pyspark.sql.functions import col, sum
df=df_csv
# Get the list of columns
columns = df.columns

# Create a dictionary to store the null count for each column
null_counts = {col_name: df.filter(col(col_name).isNull()).count() for col_name in columns}

# Convert the dictionary to a DataFrame for better display
null_counts_df = spark.createDataFrame([(k, v) for k, v in null_counts.items()], ["Column", "NullCount"])

# Show the null counts
null_counts_df.show()

+-------------------+---------+
|             Column|NullCount|
+-------------------+---------+
|              index|        0|
|        dateCrawled|      296|
|               name|        0|
|              Price|      297|
|             abtest|       30|
|        vehicleType|    37871|
| yearOfRegistration|      297|
|            gearbox|    20211|
|            powerPS|       15|
|              model|    20485|
|          kilometer|        1|
|monthOfRegistration|      297|
|           fuelType|    33388|
|              brand|       64|
|  notRepairedDamage|    72061|
|         postalCode|        1|
+-------------------+---------+



In [36]:
df.select('Price').describe().show()

+-------+------------------+
|summary|             Price|
+-------+------------------+
|  count|            371527|
|   mean|17295.124128259857|
| stddev|3587958.5758335786|
|    min|               0.0|
|    max|      2.14748365E9|
+-------+------------------+



In [37]:
# Calculate the 5th and 95th percentiles
quantiles = df.approxQuantile("price", [0.05, 0.95], 0.01)

# Extract the 5th and 95th percentiles
quantile_05 = quantiles[0]
quantile_95 = quantiles[1]

print(f"5th percentile: {quantile_05}")
print(f"95th percentile: {quantile_95}")

5th percentile: 250.0
95th percentile: 19500.0


In [38]:
df_copy=df_csv

In [39]:
df_copy = df_copy.filter(col("price") != 0)


In [40]:

df=df_copy
# Get the list of columns
columns = df.columns

# Create a dictionary to store the null count for each column
null_counts = {col_name: df.filter(col(col_name).isNull()).count() for col_name in columns}

# Convert the dictionary to a DataFrame for better display
null_counts_df = spark.createDataFrame([(k, v) for k, v in null_counts.items()], ["Column", "NullCount"])

# Show the null counts
null_counts_df.show()

+-------------------+---------+
|             Column|NullCount|
+-------------------+---------+
|              index|        0|
|        dateCrawled|        0|
|               name|        0|
|              Price|        0|
|             abtest|        0|
|        vehicleType|    34127|
| yearOfRegistration|        0|
|            gearbox|    17729|
|            powerPS|        0|
|              model|    18300|
|          kilometer|        0|
|monthOfRegistration|        0|
|           fuelType|    29948|
|              brand|        0|
|  notRepairedDamage|    66769|
|         postalCode|        0|
+-------------------+---------+



In [41]:
most_common_row = df_copy.groupBy("vehicleType").count().orderBy(col("count").desc()).first()
most_common = most_common_row["vehicleType"]
df_VehicleCopy = df_copy.fillna({"vehicleType": most_common}) #imputing the data with the Mode for the vehicleType

In [42]:
df=df_VehicleCopy
# Get the list of columns
columns = df.columns

# Create a dictionary to store the null count for each column
null_counts = {col_name: df.filter(col(col_name).isNull()).count() for col_name in columns}

# Convert the dictionary to a DataFrame for better display
null_counts_df = spark.createDataFrame([(k, v) for k, v in null_counts.items()], ["Column", "NullCount"])

# Show the null counts
null_counts_df.show()

+-------------------+---------+
|             Column|NullCount|
+-------------------+---------+
|              index|        0|
|        dateCrawled|        0|
|               name|        0|
|              Price|        0|
|             abtest|        0|
|        vehicleType|        0|
| yearOfRegistration|        0|
|            gearbox|    17729|
|            powerPS|        0|
|              model|    18300|
|          kilometer|        0|
|monthOfRegistration|        0|
|           fuelType|    29948|
|              brand|        0|
|  notRepairedDamage|    66769|
|         postalCode|        0|
+-------------------+---------+



In [166]:
value_counts = df_VehicleCopy.groupBy("vehicleType").count()

# Step 2: Sort by count in descending order
sorted_value_counts = value_counts.orderBy(col("count").desc())
sorted_value_counts.show()

+-----------+------+
|vehicleType| count|
+-----------+------+
|  limousine|127969|
| kleinwagen| 78209|
|      kombi| 66098|
|        bus| 29769|
|     cabrio| 22560|
|      coupe| 18487|
|        suv| 14501|
|     andere|  3155|
+-----------+------+



In [43]:
most_common_row =df_VehicleCopy.groupBy("gearbox").count().orderBy(col("count").desc()).first()
most_common = most_common_row["gearbox"]

# Step 2: Replace missing values in 'gearbox' column with the most common value
df_VehicleCopy = df_VehicleCopy.fillna({"gearbox": most_common})

In [44]:
df=df_VehicleCopy
# Get the list of columns
columns = df.columns

# Create a dictionary to store the null count for each column
null_counts = {col_name: df.filter(col(col_name).isNull()).count() for col_name in columns}

# Convert the dictionary to a DataFrame for better display
null_counts_df = spark.createDataFrame([(k, v) for k, v in null_counts.items()], ["Column", "NullCount"])

# Show the null counts
null_counts_df.show()

+-------------------+---------+
|             Column|NullCount|
+-------------------+---------+
|              index|        0|
|        dateCrawled|        0|
|               name|        0|
|              Price|        0|
|             abtest|        0|
|        vehicleType|        0|
| yearOfRegistration|        0|
|            gearbox|        0|
|            powerPS|        0|
|              model|    18300|
|          kilometer|        0|
|monthOfRegistration|        0|
|           fuelType|    29948|
|              brand|        0|
|  notRepairedDamage|    66769|
|         postalCode|        0|
+-------------------+---------+



In [45]:
most_common_row =df_VehicleCopy.groupBy("fuelType").count().orderBy(col("count").desc()).first()
most_common = most_common_row["fuelType"]

# Step 2: Replace missing values in 'gearbox' column with the most common value
df_VehicleCopy = df_VehicleCopy.fillna({"fuelType": most_common})

In [46]:
df=df_VehicleCopy
# Get the list of columns
columns = df.columns

# Create a dictionary to store the null count for each column
null_counts = {col_name: df.filter(col(col_name).isNull()).count() for col_name in columns}

# Convert the dictionary to a DataFrame for better display
null_counts_df = spark.createDataFrame([(k, v) for k, v in null_counts.items()], ["Column", "NullCount"])

# Show the null counts
null_counts_df.show()

+-------------------+---------+
|             Column|NullCount|
+-------------------+---------+
|              index|        0|
|        dateCrawled|        0|
|               name|        0|
|              Price|        0|
|             abtest|        0|
|        vehicleType|        0|
| yearOfRegistration|        0|
|            gearbox|        0|
|            powerPS|        0|
|              model|    18300|
|          kilometer|        0|
|monthOfRegistration|        0|
|           fuelType|        0|
|              brand|        0|
|  notRepairedDamage|    66769|
|         postalCode|        0|
+-------------------+---------+



In [47]:
most_common_row =df_VehicleCopy.groupBy("notRepairedDamage").count().orderBy(col("count").desc()).first()
most_common = most_common_row["notRepairedDamage"]

# Step 2: Replace missing values in 'gearbox' column with the most common value
df_VehicleCopy = df_VehicleCopy.fillna({"notRepairedDamage": most_common})

In [48]:
df=df_VehicleCopy
# Get the list of columns
columns = df.columns

# Create a dictionary to store the null count for each column
null_counts = {col_name: df.filter(col(col_name).isNull()).count() for col_name in columns}

# Convert the dictionary to a DataFrame for better display
null_counts_df = spark.createDataFrame([(k, v) for k, v in null_counts.items()], ["Column", "NullCount"])

# Show the null counts
null_counts_df.show()

+-------------------+---------+
|             Column|NullCount|
+-------------------+---------+
|              index|        0|
|        dateCrawled|        0|
|               name|        0|
|              Price|        0|
|             abtest|        0|
|        vehicleType|        0|
| yearOfRegistration|        0|
|            gearbox|        0|
|            powerPS|        0|
|              model|    18300|
|          kilometer|        0|
|monthOfRegistration|        0|
|           fuelType|        0|
|              brand|        0|
|  notRepairedDamage|        0|
|         postalCode|        0|
+-------------------+---------+



In [49]:
df_VehicleCopy.columns

['index',
 'dateCrawled',
 'name',
 'Price',
 'abtest',
 'vehicleType',
 'yearOfRegistration',
 'gearbox',
 'powerPS',
 'model',
 'kilometer',
 'monthOfRegistration',
 'fuelType',
 'brand',
 'notRepairedDamage',
 'postalCode']

In [50]:
df_day2=df_VehicleCopy

In [51]:
df=df_day2

In [52]:
from pyspark.sql.functions import col, sum
df.select('powerPS').describe().show()

+-------+------------------+
|summary|           powerPS|
+-------+------------------+
|  count|            360748|
|   mean|116.57623604288867|
| stddev|190.60860977764153|
|    min|                 0|
|    max|              9999|
+-------+------------------+



In [53]:
df=df.withColumn("powerPS", col("powerPS").cast("int")) #converting the power ps to the int

In [54]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# Calculate the 5th and 95th percentiles
quantiles = df.approxQuantile("powerPS", [0.05, 0.95], 0.01)

# Extract the 5th and 95th percentiles
quantile_05 = quantiles[0]
quantile_95 = quantiles[1]

print(f"5th percentile: {quantile_05}")
print(f"95th percentile: {quantile_95}")

5th percentile: 0.0
95th percentile: 231.0


In [55]:
df_imputed = df.withColumn("powerPS", 
                           when(col("powerPS") < 40, 40)
                           .when(col("powerPS") > 15000, 15000)
                           .otherwise(col("powerPS")))
df_imputed.show(5) #imputing the data for powerps

+-----+-----------+--------------------+-------+------+-----------+------------------+---------+-------+-----+---------+-------------------+--------+----------+-----------------+----------+
|index|dateCrawled|                name|  Price|abtest|vehicleType|yearOfRegistration|  gearbox|powerPS|model|kilometer|monthOfRegistration|fuelType|     brand|notRepairedDamage|postalCode|
+-----+-----------+--------------------+-------+------+-----------+------------------+---------+-------+-----+---------+-------------------+--------+----------+-----------------+----------+
|    0|       2016|          Golf_3_1.6|  480.0|  test|  limousine|              1993|  manuell|     40| golf|   150000|                  0|  benzin|volkswagen|             nein|     70435|
|    1|       2016|A5_Sportback_2.7_Tdi|18300.0|  test|      coupe|              2011|  manuell|    190| null|   125000|                  5|  diesel|      audi|               ja|     66954|
|    2|       2016|"Jeep_Grand_Chero...| 9800.0|  

In [56]:
df=df_imputed

In [57]:
df.select('model').distinct().count()

252

In [58]:
df_VehicleCopy=df
most_common_row =df_VehicleCopy.groupBy("model").count().orderBy(col("count").desc()).first()
most_common = most_common_row["model"]

# Step 2: Replace missing values in 'gearbox' column with the most common value
df_VehicleCopy = df_VehicleCopy.fillna({"model": most_common})

In [59]:
df=df_VehicleCopy

In [60]:

# Get the list of columns
columns = df.columns

# Create a dictionary to store the null count for each column
null_counts = {col_name: df.filter(col(col_name).isNull()).count() for col_name in columns}

# Convert the dictionary to a DataFrame for better display
null_counts_df = spark.createDataFrame([(k, v) for k, v in null_counts.items()], ["Column", "NullCount"])

# Show the null counts
null_counts_df.show()

+-------------------+---------+
|             Column|NullCount|
+-------------------+---------+
|              index|        0|
|        dateCrawled|        0|
|               name|        0|
|              Price|        0|
|             abtest|        0|
|        vehicleType|        0|
| yearOfRegistration|        0|
|            gearbox|        0|
|            powerPS|        0|
|              model|        0|
|          kilometer|        0|
|monthOfRegistration|        0|
|           fuelType|        0|
|              brand|        0|
|  notRepairedDamage|        0|
|         postalCode|        0|
+-------------------+---------+



In [61]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import when, col
df = df.withColumn("monthOfRegistration", when(col("monthOfRegistration") == 0, 1).otherwise(col("monthOfRegistration")))
#converting the month of registeration from 0 to 1

In [62]:
zero_count = df.filter(col("monthOfRegistration") == 0).count()
print(zero_count) #printing the number of time where monthofregisteration is 0

0


In [63]:
df = df.withColumn("postalCode", col("postalCode").cast("int")) #converting the postalcode to int

In [64]:
df.select('postalCode').distinct().count() #calculating the distinct value for postalcode

8146

In [65]:
df.select('gearbox').distinct().show() #getting distinct value for gearbox

+---------+
|  gearbox|
+---------+
|automatik|
|  manuell|
+---------+



In [66]:
# Rename column values
df_renamed = df.withColumn('gearbox', 
                           when(col('gearbox') == "automatik", "Automatic")
                           .when(col('gearbox') == "manuell", "Manual")
                          )
df_renamed.show(5) #translating from german to english

+-----+-----------+--------------------+-------+------+-----------+------------------+---------+-------+-----+---------+-------------------+--------+----------+-----------------+----------+
|index|dateCrawled|                name|  Price|abtest|vehicleType|yearOfRegistration|  gearbox|powerPS|model|kilometer|monthOfRegistration|fuelType|     brand|notRepairedDamage|postalCode|
+-----+-----------+--------------------+-------+------+-----------+------------------+---------+-------+-----+---------+-------------------+--------+----------+-----------------+----------+
|    0|       2016|          Golf_3_1.6|  480.0|  test|  limousine|              1993|   Manual|     40| golf|   150000|                  1|  benzin|volkswagen|             nein|     70435|
|    1|       2016|A5_Sportback_2.7_Tdi|18300.0|  test|      coupe|              2011|   Manual|    190| golf|   125000|                  5|  diesel|      audi|               ja|     66954|
|    2|       2016|"Jeep_Grand_Chero...| 9800.0|  

In [67]:
df=df_renamed
df.select('gearbox').distinct().show()

+---------+
|  gearbox|
+---------+
|Automatic|
|   Manual|
+---------+



In [68]:
df.select('vehicleType').distinct().show() #getting the distinct value for vehicleType

+-----------+
|vehicleType|
+-----------+
|      coupe|
| kleinwagen|
|        bus|
|     andere|
|  limousine|
|     cabrio|
|        suv|
|      kombi|
+-----------+



In [69]:
df_renamed = df.withColumn('vehicleType', 
                           when(col('vehicleType') == "coupe", "coupe")
                           .when(col('vehicleType') == "kleinwagen", "small_car")
                           .when(col( 'vehicleType' ) == "limousine", "sedan")
                           .when(col('vehicleType') == "cabrio", "convertible")
                           .when(col('vehicleType') == "bus", "bus")
                           .when(col('vehicleType') == "kombi", "station_wagon")
                           .when(col('vehicleType') == "andere", "Others")
                           .when(col('vehicleType') == "suv", "suv")
                          )

#translating the german to english                    
    

In [70]:
df=df_renamed
df.select('vehicleType').distinct().show()

+-------------+
|  vehicleType|
+-------------+
|        coupe|
|          bus|
|station_wagon|
|    small_car|
|          suv|
|  convertible|
|       Others|
|        sedan|
+-------------+



In [71]:
df.select('fuelType').distinct().show() #getting the Distinct fuel type 

+--------+
|fuelType|
+--------+
|  benzin|
| elektro|
|  andere|
|  diesel|
|     cng|
|  hybrid|
|     lpg|
+--------+



In [72]:
df_renamed = df.withColumn('fuelType', 
                           when(col('fuelType') == "benzin", "Petrol")
                           .when(col('fuelType') == "elektro", "Electric")
                           .when(col( 'fuelType' ) == "lpg", "LPG")
                           .when(col('fuelType') == "diesel", "Diesel")
                           .when(col('fuelType') == "cng", "CNG")
                           .when(col('fuelType') == "hybrid", "Hybrid")
                           .when(col('fuelType') == "andere", "Others")
                          )
#translating the fueltype from german to english

In [73]:
df=df_renamed
df.select('fuelType').distinct().show()

+--------+
|fuelType|
+--------+
|  Diesel|
|  Hybrid|
|     CNG|
|  Others|
|Electric|
|     LPG|
|  Petrol|
+--------+



In [74]:
#df = spark.createDataFrame(df_VehicleCopy, df_VehicleCopy.columns)
#df_VehicleCopy=df
#pandas_df = df_VehicleCopy.toPandas()

In [75]:
df_VehicleCopy.show(5)

+-----+-----------+--------------------+-------+------+-----------+------------------+---------+-------+-----+---------+-------------------+--------+----------+-----------------+----------+
|index|dateCrawled|                name|  Price|abtest|vehicleType|yearOfRegistration|  gearbox|powerPS|model|kilometer|monthOfRegistration|fuelType|     brand|notRepairedDamage|postalCode|
+-----+-----------+--------------------+-------+------+-----------+------------------+---------+-------+-----+---------+-------------------+--------+----------+-----------------+----------+
|    0|       2016|          Golf_3_1.6|  480.0|  test|      sedan|              1993|   Manual|     40| golf|   150000|                  1|  Petrol|volkswagen|             nein|     70435|
|    1|       2016|A5_Sportback_2.7_Tdi|18300.0|  test|      coupe|              2011|   Manual|    190| golf|   125000|                  5|  Diesel|      audi|               ja|     66954|
|    2|       2016|"Jeep_Grand_Chero...| 9800.0|  

In [76]:
df.select('notRepairedDamage').distinct().show() #finding the diatinct value for not repaird damage

+-----------------+
|notRepairedDamage|
+-----------------+
|             nein|
|               ja|
+-----------------+



In [77]:
df_renamed = df.withColumn('notRepairedDamage', 
                           when(col('notRepairedDamage') == "nein", "No")
                           .when(col('notRepairedDamage') == "ja", "Yes")
                          )
#translating the german to english

In [78]:
df=df_renamed
df.select('notRepairedDamage').distinct().show()

+-----------------+
|notRepairedDamage|
+-----------------+
|               No|
|              Yes|
+-----------------+



In [79]:
df.columns

['index',
 'dateCrawled',
 'name',
 'Price',
 'abtest',
 'vehicleType',
 'yearOfRegistration',
 'gearbox',
 'powerPS',
 'model',
 'kilometer',
 'monthOfRegistration',
 'fuelType',
 'brand',
 'notRepairedDamage',
 'postalCode']

In [80]:
df_with_difference = df.withColumn("Car_age", col('dateCrawled') - col("yearOfRegistration"))


In [81]:

df_with_difference.select('Car_age').show(5)
 # this contains the Index and name column data

+-------+
|Car_age|
+-------+
|     23|
|      5|
|     12|
|     15|
|      8|
+-------+
only showing top 5 rows



In [82]:
df.select('dateCrawled').show(2)

+-----------+
|dateCrawled|
+-----------+
|       2016|
|       2016|
+-----------+
only showing top 2 rows



In [83]:
from pyspark.sql.functions import col, expr
df = df.withColumn('dateString', expr("concat(dateCrawled, '-01-01')"))

# Convert the date string to DateType
df = df.withColumn('dateCrawled', col('dateString').cast('date'))

# Drop the intermediate 'dateString' column
df = df.drop('dateString')

# Show the DataFrame with the new 'dateCrawled' colum
df.describe

<bound method DataFrame.describe of DataFrame[index: string, dateCrawled: date, name: string, Price: float, abtest: string, vehicleType: string, yearOfRegistration: int, gearbox: string, powerPS: int, model: string, kilometer: int, monthOfRegistration: int, fuelType: string, brand: string, notRepairedDamage: string, postalCode: int, Car_age: int]>

In [84]:
df.select('dateCrawled').show(2)

+-----------+
|dateCrawled|
+-----------+
| 2016-01-01|
| 2016-01-01|
+-----------+
only showing top 2 rows



In [85]:
df = df.withColumn('dateString', expr("concat(yearOfRegistration, '-01-01')"))

# Convert the date string to DateType
df = df.withColumn('yearOfRegistration', col('dateString').cast('date'))

# Drop the intermediate 'dateString' column
df = df.drop('dateString')

# Show the DataFrame with the new 'dateCrawled' colum
df.describe

<bound method DataFrame.describe of DataFrame[index: string, dateCrawled: date, name: string, Price: float, abtest: string, vehicleType: string, yearOfRegistration: date, gearbox: string, powerPS: int, model: string, kilometer: int, monthOfRegistration: int, fuelType: string, brand: string, notRepairedDamage: string, postalCode: int, Car_age: int]>

In [86]:
df.select('yearOfRegistration').show(2)

+------------------+
|yearOfRegistration|
+------------------+
|        1993-01-01|
|        2011-01-01|
+------------------+
only showing top 2 rows



In [87]:
from pyspark.sql.functions import col, datediff
df_dateDiff=df
df_dateDiff = df_dateDiff.withColumn('Car_age', datediff(col('dateCrawled'), col('yearOfRegistration')))
#calculating the date difference between datecrawled and yearofregistration

In [88]:
df_dateDiff.select('Car_age').show(10) #car age comes out to in years

+-------+
|Car_age|
+-------+
|   8400|
|   1826|
|   4383|
|   5478|
|   2922|
|   7670|
|   4383|
|    730|
|   6574|
|   4383|
+-------+
only showing top 10 rows



In [89]:
df_dateDiff = df_dateDiff.withColumn('Car_age', col('Car_age') / 365)
df_dateDiff.select('Car_age').show(10)
#car age is converted to the year format

+------------------+
|           Car_age|
+------------------+
|23.013698630136986|
| 5.002739726027397|
|12.008219178082191|
|15.008219178082191|
| 8.005479452054795|
|21.013698630136986|
|12.008219178082191|
|               2.0|
| 18.01095890410959|
|12.008219178082191|
+------------------+
only showing top 10 rows



In [90]:
df_dateDiff = df_dateDiff.withColumn('Car_age', col('Car_age').cast('integer'))
df_dateDiff.select('Car_age').show(10) #converting float to integer datatype for car_age

+-------+
|Car_age|
+-------+
|     23|
|      5|
|     12|
|     15|
|      8|
|     21|
|     12|
|      2|
|     18|
|     12|
+-------+
only showing top 10 rows



In [91]:
import pyspark.sql.functions as F

# Define the number of bins
num_bins = 80

# Compute histogram
hist = df_dateDiff.select('Car_age').rdd.flatMap(lambda x: x).histogram(num_bins)

# Display histogram
for i in range(len(hist[0]) - 1):
    print(f"Bin {i+1}: ({hist[0][i]}, {hist[0][i+1]}): {hist[1][i]}")
    
#plotting the histogram for the Car_age


Bin 1: (-7988.0, -7875.45): 19
Bin 2: (-7875.45, -7762.9): 0
Bin 3: (-7762.9, -7650.35): 0
Bin 4: (-7650.35, -7537.8): 0
Bin 5: (-7537.8, -7425.25): 1
Bin 6: (-7425.25, -7312.7): 0
Bin 7: (-7312.7, -7200.15): 0
Bin 8: (-7200.15, -7087.6): 0
Bin 9: (-7087.6, -6975.05): 4
Bin 10: (-6975.05, -6862.5): 2
Bin 11: (-6862.5, -6749.95): 0
Bin 12: (-6749.95, -6637.4): 0
Bin 13: (-6637.4, -6524.85): 0
Bin 14: (-6524.85, -6412.3): 1
Bin 15: (-6412.3, -6299.75): 0
Bin 16: (-6299.75, -6187.2): 1
Bin 17: (-6187.2, -6074.65): 0
Bin 18: (-6074.65, -5962.1): 2
Bin 19: (-5962.1, -5849.55): 0
Bin 20: (-5849.55, -5737.0): 2
Bin 21: (-5737.0, -5624.450000000001): 0
Bin 22: (-5624.450000000001, -5511.9): 0
Bin 23: (-5511.9, -5399.35): 1
Bin 24: (-5399.35, -5286.8): 0
Bin 25: (-5286.8, -5174.25): 0
Bin 26: (-5174.25, -5061.700000000001): 1
Bin 27: (-5061.700000000001, -4949.15): 4
Bin 28: (-4949.15, -4836.6): 0
Bin 29: (-4836.6, -4724.05): 0
Bin 30: (-4724.05, -4611.5): 0
Bin 31: (-4611.5, -4498.950000000001

In [92]:
from pyspark.sql.functions import stddev_samp
stddev_df = df_dateDiff.agg(stddev_samp('Car_age').alias('stddev'))

# Show the result
stddev_df.show()
df_dateDiff.select('Car_age').describe().show()


+----------------+
|          stddev|
+----------------+
|82.1505593217502|
+----------------+

+-------+------------------+
|summary|           Car_age|
+-------+------------------+
|  count|            360748|
|   mean|11.548895627972989|
| stddev|  82.1505593217502|
|    min|             -7988|
|    max|              1016|
+-------+------------------+



In [94]:
from pyspark.sql.functions import when, col
df_imputed = df_dateDiff.withColumn("Car_age", 
                           when(col("Car_age") < 3, 3)
                           .when(col("Car_age") > 93, 93)
                           .otherwise(col("Car_age")))
df_imputed.show(5)
#imputing data for the Car_age

+-----+-----------+--------------------+-------+------+-----------+------------------+---------+-------+-----+---------+-------------------+--------+----------+-----------------+----------+-------+
|index|dateCrawled|                name|  Price|abtest|vehicleType|yearOfRegistration|  gearbox|powerPS|model|kilometer|monthOfRegistration|fuelType|     brand|notRepairedDamage|postalCode|Car_age|
+-----+-----------+--------------------+-------+------+-----------+------------------+---------+-------+-----+---------+-------------------+--------+----------+-----------------+----------+-------+
|    0| 2016-01-01|          Golf_3_1.6|  480.0|  test|      sedan|        1993-01-01|   Manual|     40| golf|   150000|                  1|  Petrol|volkswagen|               No|     70435|     23|
|    1| 2016-01-01|A5_Sportback_2.7_Tdi|18300.0|  test|      coupe|        2011-01-01|   Manual|    190| golf|   125000|                  5|  Diesel|      audi|              Yes|     66954|      5|
|    2| 20

In [96]:
from pyspark.sql.functions import min, max
min_max_df = df_imputed.select(min("Car_age").alias("min_value"), max("Car_age").alias("max_value"))
min_max_df.show()

+---------+---------+
|min_value|max_value|
+---------+---------+
|        3|       93|
+---------+---------+



In [106]:
df=df_imputed

In [107]:
df.printSchema()

root
 |-- index: string (nullable = true)
 |-- dateCrawled: date (nullable = true)
 |-- name: string (nullable = true)
 |-- Price: float (nullable = true)
 |-- abtest: string (nullable = true)
 |-- vehicleType: string (nullable = true)
 |-- yearOfRegistration: date (nullable = true)
 |-- gearbox: string (nullable = true)
 |-- powerPS: integer (nullable = true)
 |-- model: string (nullable = false)
 |-- kilometer: integer (nullable = true)
 |-- monthOfRegistration: integer (nullable = true)
 |-- fuelType: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- notRepairedDamage: string (nullable = true)
 |-- postalCode: integer (nullable = true)
 |-- Car_age: integer (nullable = true)



In [99]:
 df_dropped=df.drop(*[ 'index','name'])
#froppin the column for index and name

In [100]:
df_dropped.printSchema()

root
 |-- dateCrawled: date (nullable = true)
 |-- Price: float (nullable = true)
 |-- abtest: string (nullable = true)
 |-- vehicleType: string (nullable = true)
 |-- yearOfRegistration: date (nullable = true)
 |-- gearbox: string (nullable = true)
 |-- powerPS: integer (nullable = true)
 |-- model: string (nullable = false)
 |-- kilometer: integer (nullable = true)
 |-- monthOfRegistration: integer (nullable = true)
 |-- fuelType: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- notRepairedDamage: string (nullable = true)
 |-- postalCode: integer (nullable = true)
 |-- Car_age: integer (nullable = true)



In [101]:
df=df_dropped

In [108]:
columns = df.columns

# Create a dictionary to store the null count for each column
null_counts = {col_name: df.filter(col(col_name).isNull()).count() for col_name in columns}

# Convert the dictionary to a DataFrame for better display
null_counts_df = spark.createDataFrame([(k, v) for k, v in null_counts.items()], ["Column", "NullCount"])

# Show the null counts
null_counts_df.show()

+-------------------+---------+
|             Column|NullCount|
+-------------------+---------+
|              index|        0|
|        dateCrawled|        0|
|               name|        0|
|              Price|        0|
|             abtest|        0|
|        vehicleType|        0|
| yearOfRegistration|        0|
|            gearbox|        0|
|            powerPS|        0|
|              model|        0|
|          kilometer|        0|
|monthOfRegistration|        0|
|           fuelType|        0|
|              brand|        0|
|  notRepairedDamage|        0|
|         postalCode|        0|
|            Car_age|        0|
+-------------------+---------+



In [109]:
df.show(1)

+-----+-----------+----------+-----+------+-----------+------------------+-------+-------+-----+---------+-------------------+--------+----------+-----------------+----------+-------+
|index|dateCrawled|      name|Price|abtest|vehicleType|yearOfRegistration|gearbox|powerPS|model|kilometer|monthOfRegistration|fuelType|     brand|notRepairedDamage|postalCode|Car_age|
+-----+-----------+----------+-----+------+-----------+------------------+-------+-------+-----+---------+-------------------+--------+----------+-----------------+----------+-------+
|    0| 2016-01-01|Golf_3_1.6|480.0|  test|      sedan|        1993-01-01| Manual|     40| golf|   150000|                  1|  Petrol|volkswagen|               No|     70435|     23|
+-----+-----------+----------+-----+------+-----------+------------------+-------+-------+-----+---------+-------------------+--------+----------+-----------------+----------+-------+
only showing top 1 row



In [110]:
#Data Is Stored on Hive HDFS Automatically
#tablename="Project_Autos.autos"
#df.write.mode("overwrite").saveAsTable(tablename)

In [112]:
#output_path = "MainData"  # Change this to your desired output path
#df_VehicleCopy=df
#pandas_df = df_VehicleCopy.toPandas()
# Save DataFrame to CSV
#df.write.csv(output_path, header=True)

In [ ]:
#Creating Secound Columns

In [113]:
#Deleating columns for dateCrawled,name,yearOfRegistration
#df2=df

In [114]:
#df_dropped=df2.drop(*[ 'index','dateCrawled','name','yearOfRegistration'])

In [115]:
#df_dropped.columns

['Price',
 'abtest',
 'vehicleType',
 'gearbox',
 'powerPS',
 'model',
 'kilometer',
 'monthOfRegistration',
 'fuelType',
 'brand',
 'notRepairedDamage',
 'postalCode',
 'Car_age']

In [124]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, expr

# Initialize Spark session
spark = SparkSession.builder.appName("OutlierRemoval").getOrCreate()

def calculate_quantiles(spark_df):
    quantiles = {}
    for column in spark_df.columns:
        if spark_df.schema[column].dataType in ('double', 'float'):
            result = spark_df.selectExpr(f"percentile_approx({column}, array(0.1, 0.9)) as quantiles").first()
            if result and result['quantiles']:
                # Check if the quantiles list has exactly 2 elements
                if len(result['quantiles']) == 2:
                    quantiles[column] = {
                        'Q1': result['quantiles'][0],
                        'Q3': result['quantiles'][1]
                    }
                else:
                    print(f"Unexpected quantiles length for column: {column}")
                    quantiles[column] = {'Q1': None, 'Q3': None}
            else:
                print(f"Quantile calculation failed for column: {column}")
                quantiles[column] = {'Q1': None, 'Q3': None}
    return quantiles

def remove_outliers(spark_df):
    quantile_dict = calculate_quantiles(spark_df)
    
    filters = []
    for column, stats in quantile_dict.items():
        if stats['Q1'] is not None and stats['Q3'] is not None:
            Q1 = stats['Q1']
            Q3 = stats['Q3']
            IQR = Q3 - Q1
            filters.append(
                (col(column) >= (Q1 - 1.5 * IQR)) & (col(column) <= (Q3 + 1.5 * IQR))
            )
        else:
            print(f"Skipping column {column} due to missing quantiles.")
    
    if filters:
        combined_filter = filters[0]
        for f in filters[1:]:
            combined_filter = combined_filter & f

        filtered_df = spark_df.filter(combined_filter)
    else:
        print("No valid filters were applied.")
        filtered_df = spark_df

    return filtered_df

# Assuming `df_dropped` is your DataFrame
clean_user_car = remove_outliers(df_dropped)

# Show the cleaned DataFrame
clean_user_car.show()


No valid filters were applied.
+-------+-------+-------------+---------+-------+--------+---------+-------------------+--------+-------------+-----------------+----------+-------+
|  Price| abtest|  vehicleType|  gearbox|powerPS|   model|kilometer|monthOfRegistration|fuelType|        brand|notRepairedDamage|postalCode|Car_age|
+-------+-------+-------------+---------+-------+--------+---------+-------------------+--------+-------------+-----------------+----------+-------+
|  480.0|   test|        sedan|   Manual|     40|    golf|   150000|                  1|  Petrol|   volkswagen|               No|     70435|     23|
|18300.0|   test|        coupe|   Manual|    190|    golf|   125000|                  5|  Diesel|         audi|              Yes|     66954|      5|
| 9800.0|   test|          suv|Automatic|    163|   grand|   125000|                  8|  Diesel|         jeep|               No|     90480|     12|
| 1500.0|   test|    small_car|   Manual|     75|    golf|   150000|       

In [125]:
#loading  the data to the Hive wearhouse
df2=clean_user_car
tablename="Project_Autos.autos_Data_Modified"
df2.write.mode("overwrite").saveAsTable(tablename)

In [126]:
output_path = "DataFor_Analysis"  # Change this to your desired output path
#df_VehicleCopy=df
#pandas_df = df_VehicleCopy.toPandas()
# Save DataFrame to CSV
df2.write.csv(output_path, header=True)

    100% |████████████████████████████████| 256kB 391kB/s ta 0:00:01
Note: you may need to restart the kernel to use updated packages.
